# Ridge Regression - Ultimate Simplification Test

The simplification trend is VALIDATED on LB:
- [256,128,64]: LB 0.0972
- [64,32]: LB 0.0946 (2.7% better)
- [32,16]: LB 0.0932 (1.5% better)

Now try the ultimate simplification: Linear model (Ridge Regression)
- No hidden layers
- L2 regularization
- Per-target regressors
- Same features (Spange + DRFP + Arrhenius = 140 features)
- TTA for mixtures

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.set_default_dtype(torch.double)

print('Ridge Regression experiment')

In [ ]:
# Load data and features
DATA_PATH = '/home/data'

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}')

In [ ]:
# Combined Featurizer (same as before)
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[["Residence Time", "Temperature"]].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
        
        return torch.tensor(np.hstack([X_kinetic, X_spange, X_drfp]))

print(f'Feature dimension: {CombinedFeaturizer().feats_dim}')

In [ ]:
# Ridge Regression Model - Ultimate Simplification
class RidgeModel:
    def __init__(self, data='single', alpha=1.0):
        self.data_type = data
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = []  # 3 models, one per target
        self.scaler = StandardScaler()
        self.alpha = alpha
    
    def train_model(self, X_train, y_train):
        X_feat = self.featurizer.featurize(X_train).numpy()
        
        if self.data_type == 'full':
            # Data augmentation: add flipped version
            X_flip = self.featurizer.featurize(X_train, flip=True).numpy()
            X_feat = np.vstack([X_feat, X_flip])
            y_values = np.vstack([y_train.values, y_train.values])
        else:
            y_values = y_train.values
        
        # Scale features
        X_scaled = self.scaler.fit_transform(X_feat)
        
        # Train one Ridge model per target
        self.models = []
        for i in range(3):  # 3 targets
            model = Ridge(alpha=self.alpha)
            model.fit(X_scaled, y_values[:, i])
            self.models.append(model)
    
    def predict(self, X):
        X_feat = self.featurizer.featurize(X).numpy()
        
        if self.data_type == 'full':
            # TTA: average predictions from both orderings
            X_flip = self.featurizer.featurize(X, flip=True).numpy()
            X_scaled = self.scaler.transform(X_feat)
            X_flip_scaled = self.scaler.transform(X_flip)
            
            preds = []
            for model in self.models:
                pred = (model.predict(X_scaled) + model.predict(X_flip_scaled)) / 2
                preds.append(pred)
            return torch.tensor(np.column_stack(preds))
        else:
            X_scaled = self.scaler.transform(X_feat)
            preds = []
            for model in self.models:
                preds.append(model.predict(X_scaled))
            return torch.tensor(np.column_stack(preds))

print('RidgeModel defined: Linear model with L2 regularization, per-target regressors')

In [ ]:
# Run CV for single solvent
print("\n--- TASK 0: Single Solvent (24 folds) ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    model = RidgeModel(data='single', alpha=1.0)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X).numpy()
    for row_idx, row in enumerate(predictions):
        all_predictions.append({"task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]})
    all_actuals.append(test_Y.values)

submission_single = pd.DataFrame(all_predictions)
actuals_single = np.vstack(all_actuals)
preds_single = submission_single[['target_1', 'target_2', 'target_3']].values
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')

In [ ]:
# Run CV for full data
print("\n--- TASK 1: Full Data (13 folds) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    model = RidgeModel(data='full', alpha=1.0)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X).numpy()
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({"task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]})
    all_actuals_full.append(test_Y.values)

submission_full = pd.DataFrame(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
preds_full = submission_full[['target_1', 'target_2', 'target_3']].values
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')

In [ ]:
# Calculate overall MSE
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nComparison:')
print(f'  exp_007 [32,16]: CV 0.009262')
print(f'  This (Ridge): {overall_mse:.6f}')
print(f'  Change: {(overall_mse - 0.009262) / 0.009262 * 100:+.1f}%')
print(f'\nIf Ridge is worse, [32,16] is near optimal simplicity.')
print(f'If Ridge is better, linear models are sufficient for this task.')

In [ ]:
# Save submission
submission = pd.concat([submission_single, submission_full])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)
print(f'Submission saved: {submission.shape}')
print(submission.head())